In [ ]:
# Setup stuff
!git clone https://github.com/justin73939/ClauseWise.git
%cd /content/ClauseWise
!pip install -r requirements.txt


Cloning into 'ClauseWise'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 40 (delta 7), reused 39 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 12.79 KiB | 818.00 KiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/ClauseWise
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.2 MB/s eta 0:00:00
--2025-11-07 22:25:02--  https://github.com/TheAtticusProject/cuad/raw/main/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/TheAtticusProject/cuad/main/data.

In [ ]:
# Load the model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "ClauseWise/legalbert-clause-classifier"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ClauseWise/legalbert-clause-classifier and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Push model to huggingface hub
from huggingface_hub import login

# Login (only once per session)
login()

# Push to your namespace
model.push_to_hub("ClauseWise/legalbert-clause-classifier")
tokenizer.push_to_hub("ClauseWise/legalbert-clause-classifier")

In [16]:
from google.colab import files

# Upload your local CSV (downloaded from Atticus site)
uploaded = files.upload()

Saving master_clauses.csv to master_clauses.csv


In [30]:
import pandas as pd
import re

# Load the master CSV you uploaded
df = pd.read_csv("data/master_clauses.csv")
print("Shape before flattening:", df.shape)

# Flatten: one clause per row
long_rows_classif = []
long_rows_textgen = []
for _, row in df.iterrows():
    filename = row["Filename"]
    for col in df.columns:
        if col.endswith("-Answer"):  # find every label column
            category = col.replace("-Answer", "").strip()
            text_col = category
            if text_col not in df.columns:
                continue
            text = row[text_col]
            answer = row[col]

            #if pd.isna(text) or text in ([], "", None): continue

            # clean stringified lists
            if isinstance(text, str):
                text = text.strip("[]").replace("'", "").strip()
            if isinstance(answer, str):
                answer = answer.strip("[]").replace("'", "").strip()

            # Separate the 8 columns that do not have yes/no answers for the text generator
            if category in ["Document Name", "Parties", "Agreement Date", "Effective Date", "Expiration Date",
                            "Renewal Term", "Notice Period", "Governing Law", "Most Favored Nation"]:
                long_rows_textgen.append({
                  "document_name": filename,
                  "category": category,
                  "text": text,
                  "answer": answer
              })
                continue

            long_rows_classif.append({
                "document_name": filename,
                "category": category,
                "text": text,
                "answer": answer
            })

flat_df_classif = pd.DataFrame(long_rows_classif)
flat_df_textgen = pd.DataFrame(long_rows_textgen)
print("Flattened shape:", flat_df_classif.shape)

# Clean up text artifacts
def clean_clause(t):
    if pd.isna(t): return ""
    t = re.sub(r"<omitted>", " ", t)
    t = re.sub(r"\[\*+\]", " ", t)
    t = re.sub(r"_+", " ", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

flat_df_classif["text"] = flat_df_classif["text"].apply(clean_clause)
flat_df_classif["answer"] = flat_df_classif["answer"].apply(clean_clause)

flat_df_textgen["text"] = flat_df_textgen["text"].apply(clean_clause)
flat_df_textgen["answer"] = flat_df_textgen["answer"].apply(clean_clause)

# Drop empties and small fragments
flat_df_classif = flat_df_classif.dropna(subset=["text", "category"])
flat_df_classif = flat_df_classif[flat_df_classif["text"].str.len() > 5]
flat_df_classif = flat_df_classif.drop_duplicates(subset=["text", "category"])

flat_df_textgen = flat_df_textgen.dropna(subset=["text", "category"])
flat_df_textgen = flat_df_textgen[flat_df_textgen["text"].str.len() > 5]
flat_df_textgen = flat_df_textgen.drop_duplicates(subset=["text", "category"])

# Save cleaned file
flat_df_classif.to_csv("data/cuad_flattened_classification.csv", index=False)
print("Saved: data/cuad_flattened_classification.csv")
flat_df_textgen.to_csv("data/cuad_flattened_text_generation.csv", index=False)
print("Saved: data/cuad_flattened_text_generation.csv")

print(flat_df_classif.head(50))
print(flat_df_textgen.head(50))


Shape before flattening: (510, 83)
Flattened shape: (16830, 4)
Saved: data/cuad_flattened_classification.csv
Saved: data/cuad_flattened_text_generation.csv
                                         document_name  \
0    CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
6    CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
10   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
13   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
17   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
25   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
27   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
29   CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...   
33   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
40   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
43   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
44   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
46   EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...   
50   EuromediaHoldingsCorp_20070

In [28]:
import pandas as pd

df = pd.read_csv("data/cuad_flattened_classification.csv")

print("Unique categories found:", len(df["category"].unique()))
print(sorted(df["category"].unique()))

Unique categories found: 33
['Affiliate License-Licensee', 'Affiliate License-Licensor', 'Anti-Assignment', 'Audit Rights', 'Cap On Liability', 'Change Of Control', 'Competitive Restriction Exception', 'Covenant Not To Sue', 'Exclusivity', 'Insurance', 'Ip Ownership Assignment', 'Irrevocable Or Perpetual License', 'Joint Ip Ownership', 'License Grant', 'Liquidated Damages', 'Minimum Commitment', 'No-Solicit Of Customers', 'No-Solicit Of Employees', 'Non-Compete', 'Non-Disparagement', 'Non-Transferable License', 'Notice Period To Terminate Renewal', 'Post-Termination Services', 'Price Restrictions', 'Revenue/Profit Sharing', 'Rofr/Rofo/Rofn', 'Source Code Escrow', 'Termination For Convenience', 'Third Party Beneficiary', 'Uncapped Liability', 'Unlimited/All-You-Can-Eat-License', 'Volume Restriction', 'Warranty Duration']


In [29]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encode category labels as integers
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["category"])
num_labels = len(label_encoder.classes_)
print("Number of unique categories:", num_labels)
print(label_encoder.classes_[:10])  # sanity check

# Split into train/validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(),
    df["label"].tolist(),
    test_size=0.1,
    stratify=df["label"],
    random_state=42
)

print("Train size:", len(train_texts))
print("Val size:", len(val_texts))

Number of unique categories: 33
['Affiliate License-Licensee' 'Affiliate License-Licensor'
 'Anti-Assignment' 'Audit Rights' 'Cap On Liability' 'Change Of Control'
 'Competitive Restriction Exception' 'Covenant Not To Sue' 'Exclusivity'
 'Insurance']
Train size: 3369
Val size: 375
